In [6]:
import sys
sys.path.append("../src")
import pandas as pd
from collections import defaultdict
from tqdm import tqdm

from pipeline import SearchEngine

In [2]:
DATA_PATH = "../data/"
PERSON_ATTRIBUTES_PATH = DATA_PATH + "person-attributes.csv" 

In [5]:
person_attributes_df = pd.read_csv(PERSON_ATTRIBUTES_PATH)
attributes_list = person_attributes_df.columns[1:-1].tolist()
most_common_labels_10 = defaultdict(list)
for attribute in attributes_list:
    most_common_labels_10[attribute] = person_attributes_df[attribute].value_counts().index.tolist()[:10]
queries = ['person', 'woman', 'teacher', 'role model', 'professional']
lambda_list = [1, 0.3, 0.5, 0.7]

In [4]:
search_engine = SearchEngine()

Initializing Search Engine...
Loading doc augment dict...


600001it [00:00, 852341.99it/s]


Loading indexes...


100%|██████████| 200000/200000 [00:16<00:00, 12109.92it/s]


Loading ranker...
Loading categories...
Loading recognized categories...


...

Loading raw text dict...


9605it [00:00, 1051449.56it/s]


Loading cross encoder scorer...
Loading L2R ranker...
Training L2R ranker...


9604it [00:00, 757310.36it/s]
100%|██████████| 129/129 [03:32<00:00,  1.65s/it]


Training model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014025 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2390
[LightGBM] [Info] Number of data points in the train set: 9604, number of used features: 122
Search Engine initialized!


In [8]:
def rank_all():
    overall_ranks = []
    for lambda_val in lambda_list:
        for query in queries:
            res = search_engine.pipeline.query(query, mmr_lambda=lambda_val)
            for idx, (id, score) in tqdm(enumerate(res)):
                person = person_attributes_df[person_attributes_df['docid'] == id]
                if person.empty:
                    continue
                for attribute in attributes_list:
                    if person[attribute].item() in most_common_labels_10[attribute]:
                        overall_ranks.append({
                            'pipeline': 'lambda=' + str(lambda_val),
                            'query': query,
                            'attribute': attribute,
                            'label': person[attribute].item(),
                            'rank': idx + 1
                        })
    del person_attributes_df
    overall_ranks = pd.DataFrame(overall_ranks)
    return overall_ranks

In [9]:
overall_ranks = rank_all()
overall_ranks

ValueError: 74204 is not in list